# Set up

In [2]:
!pip install transformers
!pip install deep-translator
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

CPU = -1
CUDA = 0

bart_name = "valhalla/bart-large-finetuned-squadv1"
bart_large = pipeline('question-answering', model=bart_name, tokenizer=bart_name, device=CUDA)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [5]:
from deep_translator import GoogleTranslator

class QA_Model:

  def give_answer(self, context_eng: str, question: str, model)->str:
    question_eng = self.translator_rus_to_eng.translate(text=question)
    
    QA_input = {
        'question': question_eng,
        'context': context_eng
    }

    res = self.model(QA_input)
    answer = str(res["answer"])

    score = res['score']
    print("Score:", score)

    if (score < self.QA_THRESHOLD):
      answer = "Нет информации"

    if (not answer.isnumeric()):
      answer = self.translator_eng_to_rus.translate(text=answer)

    return answer


  def __init__(self, model):
    self.model = model
    self.translator_rus_to_eng = GoogleTranslator(source='ru', target='en')
    self.translator_eng_to_rus = GoogleTranslator(source='en', target='ru')

    self.MAX_CHAR_IN_TEXT = 2000
    self.QA_THRESHOLD = 0.1

## TEST

In [32]:
from tqdm import tqdm

In [66]:
class ContextTranslator:

  def translate(self, data, ind):
    for i, item in enumerate(tqdm(data[ind:])):
      print(data[i+ind]["nameK"], item["nameK"])
      if ("areal" in item.keys()):
        if (item["areal"]):
          data[i+ind]["areal"] = self.__sliced_translate(item["areal"])
      
      if ("ekologia" in item.keys()):
        if (item["ekologia"]):
          data[i+ind]["ekologia"] = self.__sliced_translate(item["ekologia"])
      
      if ("vozdelivanie" in item.keys()):
        if (item["vozdelivanie"]):
          data[i+ind]["vozdelivanie"] = self.__sliced_translate(item["vozdelivanie"])
      
      if ("resursi" in item.keys()):
        if (item["resursi"]):
          data[i+ind]["resursi"] = self.__sliced_translate(item["resursi"])
      
    return data


  def __sliced_translate(self, context):
    context_splitted = self.__split(context, ".")
    num_of_sentences = len(context_splitted)
    result = ""

    for i in range(0, num_of_sentences, self.SENT_NUM_FOR_LOOP):
      piece = ' '.join(context_splitted[i:i + self.SENT_NUM_FOR_LOOP])
      result += self.translator_rus_to_eng.translate(text=piece)

    context_eng = result
    
    return context_eng


  def __split(self, text, sep):
    res = [x+sep for x in text.split(sep) if x and len(x) > 2]
    return res


  def __init__(self):
    self.translator_rus_to_eng = GoogleTranslator(source='ru', target='en')
    self.SENT_NUM_FOR_LOOP = 5
    self.fields = ["nameK", "opisanie", "drugie_vidi", "areal", "ekologia", "resursi", "vozdelivanie", ]

# TRANSLATION

In [34]:
import json

with open('/content/dataset_clean_with_meds.json', "r") as f:
  data = json.load(f)

In [68]:
context_translator = ContextTranslator()

In [ ]:
data[87]

In [76]:
data[138]["vozdelivanie"] = data[138]["vozdelivanie"][:3000]

In [ ]:
data = context_translator.translate(data, 138)

In [10]:
import json

In [11]:
with open("/content/translated_data.json", "r") as f:
  data = json.load(f)

In [ ]:
data

In [12]:
qa_model = QA_Model(bart_large)

In [ ]:
result = []

In [7]:
def process():
  result = []

  for item in data["list"][0:10]:

    name = item["nameK"]
    is_in_pharmacopeia = "null"
    areal = "null"
    soil = "null"
    harvesting_period = "null"
    seed_soving_period = "null"

    if ("areal" in item.keys()):
      if (item["areal"]):
        areal = qa_model.give_answer(item["areal"], "In whese areals it is distributed?", bart_large)
        subject = qa_model.give_answer(item["areal"], "In whese subjects of the Russian Federation?", bart_large)
    
    if ("ekologia" in item.keys()):
      if (item["ekologia"]):
        soil = qa_model.give_answer(item["ekologia"], "What are the names of the substrate soil?", bart_large)
    
    if ("vozdelivanie" in item.keys()):
      if (item["vozdelivanie"]):
        harvesting_period = qa_model.give_answer(item["vozdelivanie"], "Когда период сбора урожая?", bart_large)
        seed_soving_period = qa_model.give_answer(item["vozdelivanie"], "Когда период посева урожая?", bart_large)
    
    if ("sirie" in item.keys()):
      if (item["sirie"]):
        is_in_pharmacopeia = "В фармакопеях не описано" in item["sirie"]
      
    if ("himicheski_sostav" in item.keys()):
      if (item["himicheski_sostav"]):
        chemical_composition = item["himicheski_sostav"].replace("Химический состав. ", "")

    plant_obj = {
        "name": name,
        "areal": areal,
        "subject": subject,
        "soil":soil,
        "harvesting_period":harvesting_period,
        "seed_soving_period": seed_soving_period,
        "is_in_pharmacopeia":is_in_pharmacopeia,
        "chemical_composition": chemical_composition
        }
    
    result.append(plant_obj)
    print(plant_obj)

In [ ]:
process()

In [ ]:
result

[{'name': 'АИР ОБЫКНОВЕННЫЙ',
  'areal': 'северо-восточная азия',
  'soil': 'стелющиеся кустарники и высокотравная растительность',
  'harvesting_period': 'null',
  'seed_soving_period': 'null',
  'is_in_pharmacopeia': False},
 {'name': 'АЛОЭ ДРЕВОВИДНОЕ',
  'areal': 'null',
  'soil': 'илистые и аллювиальные',
  'harvesting_period': 'null',
  'seed_soving_period': 'null',
  'is_in_pharmacopeia': 'null'},
 {'name': 'АЛТЕЙ ЛЕКАРСТВЕННЫЙ',
  'areal': 'null',
  'soil': 'скалистый',
  'harvesting_period': 'Нет информации',
  'seed_soving_period': 'Нет информации',
  'is_in_pharmacopeia': False},
 {'name': 'АММИ БОЛЬШАЯ',
  'areal': 'Европейская часть России',
  'soil': 'пески',
  'harvesting_period': 'Нет информации',
  'seed_soving_period': 'Нет информации',
  'is_in_pharmacopeia': 'null'},
 {'name': 'АММИ ЗУБНАЯ',
  'areal': 'null',
  'soil': 'пески',
  'harvesting_period': 'Нет информации',
  'seed_soving_period': 'Нет информации',
  'is_in_pharmacopeia': False},
 {'name': 'АНИС ОБЫКНОВЕ

In [ ]:
with open("./output_data.json", "w", encoding="utf-8") as f:
    f.write(json.dumps({"obj_list": result}, ensure_ascii=False))